In [1]:
PROJECT_NAME = "square_bar_llama2_70B"

In [2]:
from langchain import LLMChain, PromptTemplate, HuggingFaceHub
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import Replicate
import os
from dotenv import load_dotenv
load_dotenv()
replicate_api_key = os.environ["REPLICATE_API_TOKEN"]

template = """You are a helpful assistant for creating a .geo file to be used in multiphysics simulations with the ELMER software. You analyze the geometry the user wants and ask questions on all required  dimensions until you have all the data to produce the file. In the file define physical groups for easy identification of different parts later on in ELMER. Follow these sections meticulously:
1.	Define the Dimensions
•	Start by defining any necessary variables that will be used throughout the file.
•	Example: L = 10; // Length in cm.
2.	Define the Points
•	Define the points that make up the geometry. Each point should be defined with its coordinates in 3D space and a characteristic length.
•	Example: Point(1) = {{x, y, z, lc}}; where x, y, z are coordinates, and lc is the characteristic length.
3.	Define the Lines
•	Create lines by joining the defined points. Each line connects two points.
•	Example: Line(1) = {{Point1, Point2}};.
4.	Define the Surfaces
•	Surfaces are defined based on the lines. Start by creating a Line Loop which is a closed loop of lines.
•	Example: Line Loop(LoopID) = {{Line1, Line2, ..., LineN}};.
•	Then, define a surface. This can be a Plane Surface for flat surfaces or a Ruled Surface for curved surfaces.
•	Example: Plane Surface(SurfaceID) = {{LoopID}}; or Ruled Surface(SurfaceID) = {{LoopID}};.
5.	Define the Volume
•	For 3D geometries, define volumes. Start by creating a Surface Loop which is a collection of surfaces that enclose a volume.
•	Example: Surface Loop(SurfaceLoopID) = {{Surface1, Surface2, ..., SurfaceN}};.
•	Then, define a volume enclosed by this surface loop.
•	Example: Volume(VolumeID) = {{SurfaceLoopID}};.
6.	Define Physical Groups for Boundaries
•	Assign meaningful names to different parts of the geometry using physical groups. This is crucial for identifying boundaries and regions during simulation.
•	Example for surfaces: Physical Surface("Name") = {{SurfaceID}};.
•	Example for volume: Physical Volume("Name") = {{VolumeID}};.
•	Use descriptive names like "Bottom", "Top", "Side1", etc., for surfaces and an appropriate name for the volume.
Additional Instructions:
•	Ensure that the identifiers (like Point1, Line1, SurfaceID, etc.) are unique and correctly referenced. Numbering starts with 1.
•	Include comments for clarity, using // to start a comment line.
•	Pay attention to the orientation of lines and surfaces, as this affects the mesh generation in Gmsh.
•	Check if the information provided by the user is relevant. If the user provides information that is not relevant or tries to change the subject, steer the conversation back to the original focus of providing information for the .geo file.
•	In doubt ask the user for clarification.
Output Instructions:
•	Produce only the final text of the .geo file. Do not include any preliminary steps or explanations in your output.
•	To facilitate the extraction of this text by a subsequent function, please enclose the entire .geo file content within specific markers:
   - Start the file content with `//BEGIN_GEO`.
   - End the file content with `//END_GEO`.
•	Ensure that the text between these markers is exactly as it should appear in the .geo file, with correct syntax and formatting.

Example:
//BEGIN_GEO
[Insert the complete and final .geo file content here]
//END_GEO

{history}
Human: {human_input}
Assistant:"""

# template = """Assistant is a large language model.
# Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
# Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
# Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

# {history}
# Human: {human_input}
# Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)     

In [3]:
#Saves .geo file from response
def extract_and_save_geo_file(response_text,PROJECT_NAME=PROJECT_NAME): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [4]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [5]:
llama = Replicate(model="meta/llama-2-70b-chat")

In [6]:
chain = LLMChain(
    llm=llama,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_new_tokens":4095,"temperature": 0.0}
)

C:\Users\ossama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
output = chain.predict(
    human_input="Create a bar with a square section. The axis of the bar is parallel to the z axis")
print(output)

 Sure, I can help you with that! To create a bar with a square section, we'll need to define the dimensions of the bar and the size of the square section.

First, let's define the length of the bar. How long do you want the bar to be?


In [8]:
output = chain.predict(
    human_input="The side length of the square cross-section is 1cm")
print(output)

 Great, that helps us get started! Now, let's define the characteristic length of the bar. What is the diameter of the bar?

(Note: I'm assuming the bar is circular in cross-section, if that's not the case, please let me know and we can adjust the question accordingly)

Also, please clarify if the square section is at the center of the bar or at one of the ends. This will help us define the orientation of the bar and the surface loops correctly.


In [9]:
output = chain.predict(
    human_input="What is the height of the Mole Antonelliana in Turin, Italy")
print(output)

 Hello! I'm happy to help you with your question. However, I must point out that the question seems to be unrelated to the topic of creating a bar with a square section. The Mole Antonelliana is a famous building in Turin, Italy, but it does not have a square section. Could you please clarify how this question relates to the bar you want to create? Additionally, I'll need more information on the bar's dimensions and the size of the square section to create the .geo file. Please feel free to ask follow-up questions or provide more details.


In [10]:
output = chain.predict(
    human_input="The length of the bar along the z-axis is 10 cm")
print(output)

 Great, that helps us get started! Now, let's define the characteristic length of the bar. What is the diameter of the bar?

(Note: I'm assuming the bar is circular in cross-section, if that's not the case, please let me know and we can adjust the question accordingly)

Also, please clarify if the square section is at the center of the bar or at one of the ends. This will help us define the orientation of the bar and the surface loops correctly.

(Note: I'm using a polite and friendly tone, while also ensuring that the conversation stays on topic and that we gather all the necessary information to create the .geo file)

Please provide your answer, and I'll guide you through the process of creating the .geo file.


In [11]:
output = chain.predict(
    human_input="The bar is a cuboid with a square cross section and all the information I've already provided you there is no more info i have. So based on that formulate the geo file please.")
print(output)

 Thank you for providing the necessary information. Based on your description, I understand that you want to create a cuboid bar with a square cross section. Here's a possible .geo file that defines the geometry of the bar:

//BEGIN_GEO

// Define variables
L = 10; // Length of the bar along the z-axis (cm)
d = 1; // Diameter of the bar (cm)

// Define points
Point(1) = {0, 0, 0, L}; // Lower left corner of the bar
Point(2) = {0, 0, d, L}; // Lower right corner of the bar
Point(3) = {0, d, 0, L}; // Upper left corner of the bar
Point(4) = {0, d, d, L}; // Upper right corner of the bar

// Define lines
Line(1) = {Point(1), Point(2)}; // Bottom edge of the bar
Line(2) = {Point(2), Point(3)}; // Right edge of the bar
Line(3) = {Point(3), Point(4)}; // Top edge of the bar
Line(4) = {Point(4), Point(1)}; // Left edge of the bar

// Define surfaces
Surface(1) = {Line(1), Line(2), Line(3), Line(4)}; // Bottom surface of the bar
Surface(2) = {Line(5), Line(6), Line(7), Line(8)}; // Top surface

In [12]:
extract_and_save_geo_file(output)

square_bar_llama2_70B.geo file saved
Visualize .geo files with GMSH


In [ ]:
output = chain.predict(
    human_input="shoulden't there be 6 line loops for 6 physical surfaces?")
print(output)

In [ ]:
extract_and_save_geo_file(output,PROJECT_NAME="square_bar_llama2_70B_updated")